In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

import numpy as np
import os, json
import cv2

import torch
from torchvision import models
import torch.nn as nn
#transforms
#from torchvision.transforms import Compose, Normalize, ToTensor
from torch.autograd import Variable
import torch.nn.functional as F
#from facenet_pytorch import InceptionResnetV1


import argparse


from sklearn.cluster import KMeans

from pytorch_grad_cam import (
    GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus,
    AblationCAM, XGradCAM, EigenCAM, EigenGradCAM,
    LayerCAM, FullGrad, GradCAMElementWise
)
from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import (
    show_cam_on_image, deprocess_image, preprocess_image
)
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from flashtorch.utils import apply_transforms, load_image
from torchvision import transforms




In [ ]:
image_path = "Celeba_spoof/test/fake/494514.png"
#image_path = "meeting_3/correct_prediction/4/494684.png"
rgb_img_original = cv2.imread(image_path, 1)[:, :, ::-1]
plt.imshow(rgb_img_original)
rgb_img = np.float32(rgb_img_original) / 255

In [ ]:
# resize and take the center part of image to what our model expects
def get_input_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])       
    transf = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize
    ])    

    return transf

def get_input_tensors(img):
    transf = get_input_transform()
    # unsqeeze converts single image to batch of 1
    return transf(img).unsqueeze(0)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
print(device)
face_model = "models/facePADceleb_adamW_new.pt"
model = torch.load(face_model)
#print(model)
model.to(device)


In [ ]:
rgb_img_pil = Image.fromarray(rgb_img_original)
img_t = get_input_tensors(rgb_img_pil)
#img_t = img_t.cuda()
#print(img_t.device)
model.eval()

logits = model(img_t.cuda())


In [ ]:
probs = F.softmax(logits, dim=1)
probs2 = probs.topk(2)

In [ ]:
print(probs2)

In [ ]:
def apply_threshold(cam, threshold):
    return np.where(cam >= threshold, cam, 0)

In [ ]:
target_class = probs2.indices[0, 0].item() 

target_layers = [model.features.denseblock4.denselayer24.conv2]
targets = [ClassifierOutputTarget(target_class)]

cam1 = GradCAM(model=model, target_layers=target_layers)
grayscale_cam1 = cam1(input_tensor=img_t.cuda(), targets=targets)
grayscale_cam1 = grayscale_cam1[0, :]

print("cam1", grayscale_cam1)
print(grayscale_cam1.shape)

min_val1 = grayscale_cam1.min()    
max_val1 = grayscale_cam1.max()
print("max_gradcam:", max_val1)
print("min_gradcam:", min_val1)
if max_val1 > 0:  # Avoid division by zero
    grayscale_cam1_norm = grayscale_cam1 / max_val1
else:
    grayscale_cam1_norm = grayscale_cam1  # Keep it unchanged if max is zero
    #grayscale_cam1_norm = grayscale_cam1 / grayscale_cam1.max()
    
cam2 = HiResCAM(model=model, target_layers=target_layers)
grayscale_cam2 = cam2(input_tensor=img_t.cuda(), targets=targets)
grayscale_cam2 = grayscale_cam2[0, :]

print("cam2", grayscale_cam2)
print(grayscale_cam2.shape)
    
min_val2 = grayscale_cam2.min()
max_val2 = grayscale_cam2.max()
print("max_hires:", max_val2)
print("min_hires:", min_val2)
if max_val2 > 0:  # Avoid division by zero
    grayscale_cam2_norm = grayscale_cam2 / max_val2
else:
    grayscale_cam2_norm = grayscale_cam2  # Keep it unchanged if max is zero
    #grayscale_cam2_norm = grayscale_cam2 / grayscale_cam2.max()
    
cam3 = GradCAMPlusPlus(model=model, target_layers=target_layers)
grayscale_cam3 = cam3(input_tensor=img_t.cuda(), targets=targets)
grayscale_cam3 = grayscale_cam3[0, :]
    
print("cam3", grayscale_cam3) 
print(grayscale_cam3.shape)

min_val3 = grayscale_cam3.min() 
max_val3 = grayscale_cam3.max()
print("max_++:", max_val3)
print("min_++:", min_val3)
if max_val3 > 0:  # Avoid division by zero
    grayscale_cam3_norm = grayscale_cam3 / max_val3
else:
    grayscale_cam3_norm = grayscale_cam3  # Keep it unchanged if max is zero
    #grayscale_cam3_norm = grayscale_cam3 / grayscale_cam3.max()

grayscale_cam = (grayscale_cam1_norm + grayscale_cam2_norm + grayscale_cam3_norm) / 3
    
print("cam",grayscale_cam)
  
threshold_1 = np.percentile(grayscale_cam, 90)
print("threshold", threshold_1)

grayscale_cam_avg = apply_threshold(grayscale_cam, threshold_1)
print(grayscale_cam_avg.shape)

grayscale_cams = [grayscale_cam_avg, grayscale_cam1, grayscale_cam2, grayscale_cam3]

for i, grayscale_cam in enumerate(grayscale_cams):
    cam_image = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
    cam_image = cv2.cvtColor(cam_image, cv2.COLOR_RGB2BGR)
    
    cam_output_path = os.path.join('output/cams', f'cam_{i+1}.jpg')
    cv2.imwrite(cam_output_path, cam_image)



In [ ]:
import cv2

def apply_mask(image, heatmap, mask_type='black'):
    """
    Mask the image based on the Grad-CAM heatmap.
    Arguments:
    - image: Original input image (numpy array)
    - heatmap: Grad-CAM heatmap (numpy array)
    - mask_type: Type of masking ('black', 'blur', or 'mean')
    """
     
    threshold = np.percentile(heatmap, 0)
    print("threshold", threshold)
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))  # Resize heatmap to image size
    mask = heatmap > threshold  # Create a binary mask of important regions
    
    
    
    # Apply masking
    perturbed_image = image.copy()
    if mask_type == 'black':
        perturbed_image[mask] = 0  # Set important regions to black
    elif mask_type == 'mean':
        mean_pixel_value = np.mean(image, axis=(0, 1), keepdims=True)
        perturbed_image[mask] = mean_pixel_value  # Set important regions to mean pixel value
    elif mask_type == 'blur':
        blurred = cv2.GaussianBlur(image, (21, 21), 0)
        perturbed_image[mask] = blurred[mask]  # Apply blur to important regions
    
    pil_image = Image.fromarray(perturbed_image.astype('uint8'))
    return pil_image


In [ ]:
perturbed_image = apply_mask(rgb_img_original, grayscale_cam_avg)

plt.imshow(perturbed_image)



perturbed_image_t = get_input_tensors(perturbed_image)

In [ ]:
def shift_mask(mask, shift_y, shift_x):
    """
    Shifts the mask by given offsets. Wraps around if necessary.
    
    Args:
        mask: Input binary mask (2D array).
        shift_y: Vertical shift (positive for down, negative for up).
        shift_x: Horizontal shift (positive for right, negative for left).
    
    Returns:
        Shifted mask of the same size.
    """
    # Perform shift with wrapping using np.roll
    shifted_mask = np.roll(mask, shift_y, axis=0)  # Shift vertically
    shifted_mask = np.roll(shifted_mask, shift_x, axis=1)  # Shift horizontally
    return shifted_mask

def apply_mask_random(image, heatmap, mask_type='black'):
    """
    Mask the image based on the Grad-CAM heatmap.
    Arguments:
    - image: Original input image (numpy array)
    - heatmap: Grad-CAM heatmap (numpy array)
    - mask_type: Type of masking ('black', 'blur', or 'mean')
    """
     
    threshold=np.percentile(heatmap, 0)
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))  # Resize heatmap to image size
    mask = heatmap > threshold  # Create a binary mask of important regions
    
    # Generate random shifts
    np.random.seed(42)
    #shift_y = np.random.randint(-mask.shape[0] // 2, mask.shape[0] // 2)
    #shift_x = np.random.randint(-mask.shape[1] // 2, mask.shape[1] // 2)
    
    shift_y = np.random.randint(-image.shape[0], image.shape[0])
    shift_x = np.random.randint(-image.shape[1], image.shape[1])
    
    print("Vertical:", shift_y)
    print("Horizontal:", shift_x)

    # Shift the mask
    shifted_mask = shift_mask(mask, shift_y, shift_x)
    
    # Apply masking
    perturbed_image = image.copy()
    if mask_type == 'black':
        perturbed_image[shifted_mask] = 0  # Set important regions to black
    elif mask_type == 'mean':
        mean_pixel_value = np.mean(image, axis=(0, 1), keepdims=True)
        perturbed_image[mask] = mean_pixel_value  # Set important regions to mean pixel value
    elif mask_type == 'blur':
        blurred = cv2.GaussianBlur(image, (21, 21), 0)
        perturbed_image[mask] = blurred[mask]  # Apply blur to important regions
    
    pil_image = Image.fromarray(perturbed_image.astype('uint8'))
    return pil_image


In [ ]:
random_image = apply_mask_random(rgb_img_original, grayscale_cam_avg)
plt.imshow(random_image)

random_image_t = get_input_tensors(random_image)

In [ ]:
target_class = probs2.indices[0, 0].item() 
print(target_class)

In [ ]:
def evaluate_confidence_drop(model, device, original_image, perturbed_image, target_class):
    """
    Measure the drop in confidence between the original and perturbed image.
    Arguments:
    - model: Trained model
    - original_image: Original input image (tensor)
    - perturbed_image: Image with masked important regions (tensor)
    - target_class: Target class for the prediction
    """
    model.to(device)
    model.eval()

    # Original confidence
    original_output = model(original_image.cuda())
    original_confidence = torch.softmax(original_output, dim=1)[0, target_class].item()

    # Perturbed confidence
    perturbed_output = model(perturbed_image.cuda())
    perturbed_confidence = torch.softmax(perturbed_output, dim=1)[0, target_class].item()

    # Calculate confidence drop
    confidence_drop = original_confidence - perturbed_confidence
    return confidence_drop

In [ ]:
Ensemble_confidence_drop = evaluate_confidence_drop(model, device, img_t, perturbed_image_t, target_class)
print(Ensemble_confidence_drop)

In [ ]:
random_confidence_drop = evaluate_confidence_drop(model, device, img_t, random_image_t, target_class)
print(random_confidence_drop)

In [ ]:
def only_mask(image, heatmap):
    """
    Mask the image based on the Grad-CAM heatmap.
    Arguments:
    - image: Original input image (numpy array)
    - heatmap: Grad-CAM heatmap (numpy array)
    - mask_type: Type of masking ('black', 'blur', or 'mean')
    """
    threshold = np.percentile(heatmap, 0)
    # Convert to NumPy array
    #image = np.array(image)

    # Resize heatmap to match the image size
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))

    # Create a binary mask
    mask = heatmap > threshold

    # Ensure the image is a proper copy
    perturbed_image = np.copy(image)

    # Initialize the result image with black
    result = np.zeros_like(perturbed_image)

  

    # Copy the mask region to the result
    result[mask] = perturbed_image[mask]
    
    
    pil_image = Image.fromarray(result.astype('uint8'))
    return pil_image


In [ ]:
def only_mask_random(image, heatmap):
    """
    Mask the image based on the Grad-CAM heatmap.
    Arguments:
    - image: Original input image (numpy array)
    - heatmap: Grad-CAM heatmap (numpy array)
    - mask_type: Type of masking ('black', 'blur', or 'mean')
    """
     
    threshold = np.percentile(heatmap, 0)
    # Convert the PIL image to a numpy array
    #image = np.array(image)
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))  # Resize heatmap to image size
    mask = heatmap > threshold  # Create a binary mask of important regions
    
    # Generate random shifts
    np.random.seed(42)
    #shift_y = np.random.randint(-mask.shape[0] // 2, mask.shape[0] // 2)
    #shift_x = np.random.randint(-mask.shape[1] // 2, mask.shape[1] // 2)
    
    shift_y = np.random.randint(-image.shape[0], image.shape[0])
    shift_x = np.random.randint(-image.shape[1], image.shape[1])
    

    # Shift the mask
    shifted_mask = shift_mask(mask, shift_y, shift_x)
    
     # Ensure the image is a proper copy
    perturbed_image = np.copy(image)

    # Initialize the result image with black
    result = np.zeros_like(perturbed_image)

  

    # Copy the mask region to the result
    result[shifted_mask] = perturbed_image[shifted_mask]
    
    
    pil_image = Image.fromarray(result.astype('uint8'))
    return pil_image



In [ ]:
mask_image = only_mask(rgb_img_original, grayscale_cam_avg)

plt.imshow(mask_image)



mask_image_t = get_input_tensors(mask_image)

In [ ]:
random_mask_image = only_mask_random(rgb_img_original, grayscale_cam_avg)
plt.imshow(random_mask_image)

random_mask_image_t = get_input_tensors(random_mask_image)

In [ ]:
Ensemble_mask_confidence_drop = evaluate_confidence_drop(model, device, img_t, mask_image_t, target_class)
print(Ensemble_mask_confidence_drop)

In [ ]:
random_mask_confidence_drop = evaluate_confidence_drop(model, device, img_t, random_mask_image_t, target_class)
print(random_mask_confidence_drop)